<img style="float: left; margin: 30px 15px 15px 15px;" src="https://pngimage.net/wp-content/uploads/2018/06/logo-iteso-png-5.png" width="300" height="500" /> 
    
    
### <font color='navy'> Simulación de Procesos Financieros. 

**Nombres:** Diego Alberto López Lazareno (Estudiante 1) y Esteban Márquez Delgado (Estudiante 2).

**Fecha:** 14 de octubre del 2021.

**Expediente** : IF722100, IF700637.
**Profesor:** Oscar David Jaramillo Zuluaga.
    
**Link Github**: https://github.com/diegolazareno/Proyecto_DALopez_EMarquez

# Proyecto TEMA-2

# 1.1 Valuación Intrínseca Probabilista:
## O Cómo Valorar Empresas Financieras Con Simulación Montecarlo

## 1.2 Objetivos

### 1.2.1 Objetivo General

+ Obtener una distribución de probabilidad para el valor intrínseco de las empresas financieras que forman parte del índice S&P $500$.

### 1.2.2 Objetivos Específicos

+ Descargar, con ayuda de la API de Financial Modeling Prep, los fundamentales históricos de las empresas del sector financiero que están listadas en el índice S&P $500$.
+ Obtener la distribución de probabilidad de cada nodo que simularemos a través de la manipulación aritmética de los fundamentales históricos descargados.
+ Generar variables aleatorias que sigan la distribución de probabilidad de su respectivo nodo con las técnicas aprendidas durante el presente curso de *Simulación de Procesos Financieros*.
+ Valorar a determinada(s) empresa(s) del sector financiero a través del *Modelo de Dividendos Descontados*, usando como parámetros de dicho método de valuación intrínseca a las variables aleatorias generadas de cada nodo. Este proceso nos arrojará una distribución de probabilidad para el valor intrínseco de la firma en cuestión.
+ Interpretar los resultados obtenidos en el punto anterior.


## 1.3 Definición

### 1.3.1 Escenario

Se modelará la valoración de acciones de empresas que pertenecen al sector financiero a través del **Modelo de Dividendos Descontados**. Esta técnica de valoración consiste en descontar o *traer a valor presente* los futuros flujos de efectivo, en este caso los dividendos, que se esperan obtener por la tenencia de una acción. El descuento de dichos flujos de efectivo esperados se conoce como **valor intrínseco**: esto representa la cantidad máxima que debería pagar un inversor por una participación en un negocio si considera a sus pronósticos como suficientemente certeros. 

Como se mencionó, es a través del Modelo de Dividendos Descontados que se puede conocer el valor intrínseco de un negocio, o bien, de sus acciones, pues éstas representan una participación dentro del mismo. Tal modelo suele ser empleado para valorar empresas que pertenecen al sector financiero, por lo que en este proyecto se valorarán únicamente empresas financieras (bancos comerciales, aseguradoras, bancos de inversión, etc.) que pertenezcan al índice S&P $500$ (actualmente son cerca de $65$ empresas). Lo interesante de esta técnica de valoración es que necesita de varias entradas (inputs) para pronosticar el crecimiento de los dividendos y su posterior descuento. Aquí es donde entran las herramientas que hemos aprendido durante el curso: tomaremos de una base de datos, [Financial Modeling Prep](https://financialmodelingprep.com/developer/docs), los fundamentales de estas empresas financieras durante varios ejercicios fiscales para simular los distintos nodos que consideramos necesarios; de esta forma estamos llevando la valoración fundamental a un terreno probabilista, donde las distribuciones de características de las demás firmas servirán como entradas (inputs) para el modelo. 

Lo que buscamos con lo anteriormente descrito es obtener una distribución de probabilidad que refleje el valor intrínseco de una acción para distintos escenarios, donde en cada uno de estos los fundamentales del negocio cambiarán al largo plazo (en torno al año $5$ de valoración); adquiriendo así características de otras firmas, dadas por la distribución de probabilidad de fundamentales del conjunto de firmas que pertenecen al sector financiero. La principal motivación de esta aventura es que como inversores nos es más útil una distribución de probabilidad para el valor intrínseco de un activo, ya que de esta forma se captura la incertidumbre del mundo cambiante en el que estas empresas desenvuelven sus operaciones.

### 1.3.2 Diagrama

<img src = "Diagrama.png" width = "802px" height = "395px">

## 1.4 Nodos


El **Modelo de Dividendos Descontados** necesita de proyecciones, en específico para los dividendos que se esperan obtener por la tenencia de una acción. Las firmas financieras suelen pagar dividendos a sus accionistas, por lo que este modelo nos es útil para determinar un valor intrínseco para tales empresas. Para llegar a los dividendos que se reparten a los accionistas hay que empezar por pronosticar las ventas de la empresa. Una vez hecho esto se deben conocer los ingresos netos que tendrá la empresa, para esto es útil el margen de beneficio neto; nuestro primer nodo a simular:

### 1.4.1 Margen de Beneficio Neto 

El **Margen de Beneficio Neto** es el porcentaje de las ventas que queda a una empresa después de las erogaciones originadas por los costos, pagos de intereses e impuestos. Matemáticamente se define como:

$$ Margen \space de \space Beneficio \space Neto = \frac{Utilidad \space Neta}{Ventas} $$

Con ayuda de la base de datos (que proporciona fundamentales de todas las firmas pertenecientes al sector financiero) obtendremos una distribución de probabilidad para el margen de beneficio neto, de esta forma podremos simular distintos escenarios para la valoración de una firma en específico, donde en un periodo futuro sus fundamentales se desvían hacia alguna variable aleatoria, algún otro margen de beneficio neto, que se encuentre en la distribución de probabilidad. Reflejando así el que una empresa con un amplio margen puede caer en un margen neto mediocre, y viceversa, que una empresa con márgenes mediocres puede gozar en un futuro de jugosos márgenes.

### 1.4.2 Ratio Payout

Una vez se tienen los beneficios netos el consejo de administración decide la repartición de dividendos a los accionistas. El **Ratio Payout** nos ayuda a cuantificar qué proporción de tales ingresos se reparten a los tenedores de acciones. Matemáticamente se define como:

$$ Ratio \space Payout = \frac{Dividendos}{Utilidad \space Neta} $$

A través de la simulación tendremos distintos escenarios para una etapa futura en la valoración, donde el Ratio Payout de cualquier firma se desvía hacia otro Ratio Payout, esto lograría capturar que, dependiendo del ciclo económico, se aumenta o recorta el reparto de dividendos a los accionistas.


### 1.4.3 Precio a Dividendos

El ratio **Precio a Dividendos** es simplemente un múltiplo que indica a un inversor cuánto está pagando por cada unidad de dividendos que se reparten al momento que adquiere una acción. Matemáticamente se define como: 

$$ Precio \space a \space Dividendos = \frac{Precio \space Por \space Acción}{Dividendos \space Por \space Acción} $$

Tal múltiplo nos ayudará a determinar un valor terminal para la acción que estemos valorando. Este nodo es importante porque no podemos seguir pronosticando dividendos perpetuamente, por ello es necesario un valor terminal que refleje el precio hipotético que se obtendría para la acción al final de la valoración. Esto se logra con el producto entre el ratio Precio a Dividendos y el último dividendo estimado.

### 1.4.4 Beta

Una vez hechos los pronósticos es necesario descontarlos a valor presente. Para ello se empleará el modelo $CAPM$, donde uno de sus inputs es la **Beta**, la cual nos ayuda a cuantificar el riesgo sistémico o de mercado. Para nuestros escenarios de valoración descontaremos los flujos de efectivo esperados con distintas tasas, que provendrán de la distribución de probabilidad para las betas de todas las firmas pertenecientes al sector financiero. 

**La suma de todos los dividendos descontados es el valor intrínseco para la firma en cuestión en el escenario de valoración i-ésimo.**


## 1.5 Hipótesis & Supuestos

### 1.5.1 Definición De Hipótesis

> El objetivo último de la valuación de activos financieros para un inversor es tomar ventaja si existe una discrepancia significativa en el precio al que cotiza la acción y su valor intrínseco. Nosotros obtendremos una distribución de probabilidad para el valor intrínseco de determinada compañía, por lo que podemos calcular el valor esperado de tal distribución (su media). Entonces podemos preguntarnos lo siguiente: **¿la media de la distribución de probabilidad del valor intrínseco es igual al último precio de cotización?** Si esto se cumple entonces significaría que el activo está valuado justamente por los mercados, pues cotiza a su valor intrínseco esperado; de lo contrario, si esto no se cumple significaría que hay una ineficiencia, pues el activo no está valuado justamente por los mercados, por lo que podemos tomar una ventaja. Esta pregunta puede traducirse a una prueba de hipótesis, que se formularía de la siguiente manera:
>
>$$ H_o : \mu = S_o $$
>$$ H_a : \mu \neq S_o $$
>
>donde $S_o$ es el último precio de cotización.
>
> Lo que estamos buscando son activos donde se rechace esta hipótesis, es decir, donde exista ineficiencias y los mercados no los estén valuando correctamente.

### 1.5.2 Definición De Supuestos

> 1. Suponemos que se puede determinar un valor intrínseco, al menos lo suficientemente certero, para las firmas pertenecientes al sector financiero a través del Modelo de Dividendos Descontados. 

> 2. A su vez, al emplear estos métodos de valoración, suponemos que los mercados no son completamente eficientes; ya que de serlo no sería necesaria una valoración, pues el precio al que se negocian las acciones reflejaría el valor del negocio a cada momento. 

> 3. Otro gran supuesto es que los fundamentales de cada firma en lo individual podrán experimentar, en un momento futuro, desviaciones importantes hacia los fundamentales de otras firmas, reflejando así los distintos ciclos de negocio: una firma con un éxito extraordinario podría volverse un negocio mediocre, y viceversa, una firma mediocre, con el tiempo, podría volverse un negocio extraordinario. Este supuesto es el que conforma la piedra angular de este proyecto; pues hay que recordar que obtendremos distribuciones de probabilidad con los fundamentales de todas las firmas del sector financiero para llevar a cabo las valoraciones.

> 4. Suponemos que las distribuciones de probabilidad de los nodos que simularemos son independientes entre sí.

## 1.6 Obtención De Bases De Datos

In [ ]:
# Librerías necesarias
# Análisis y manejo de datos
import pandas as pd
import numpy as np

# Yfinance
import yfinance as yf

# Conexión a la API de Financial Modeling Prep
from pyfmpcloud import settings
from pyfmpcloud import company_valuation as cv
api_key = pd.read_csv("Data/apiKey_FMP.csv").iloc[0, 0]
settings.set_apikey(api_key)

# Filtrado de alertas
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def boxplot(data, size, color, title):
    """
    boxplot grafica un diagrama de caja para representar los datos ingresados.

    *data : datos numéricos.
    *size : tamaño del gráfico, se debe de ingresar como una tupla de la forma (ancho, alto).
    *color : color del gráfico.
    *título : título del gráfico.

    """

    plt.figure(figsize = (size[0], size[1]))
    plt.title(title)
    sns.boxplot(x = data, color = color)
    
def logTransformation(data : "Datos"):
    """
    logTransformation efectúa una transformación logarítmica a los datos ingresados para tratar de reducir su sesgo.

    *data : son los datos a transformar.

    """

    return np.log(data)

## 1.7 Simulación

## 1.8 Visualización De Resultados De Simulación

## 1.9 Conclusiones

## 1.10 Referencias Consultadas